In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/MyDrive/git_commit/energy

/content/drive/MyDrive/git_commit/energy


In [ ]:
!git config --global user.email 'vllkey@naver.com'
!git config --global user.name 'noweahc'